<a href="https://colab.research.google.com/github/aditeyabaral/gpt2-implementation/blob/main/openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI based Implementation

[Article](https://medium.com/@ngwaifoong92/beginners-guide-to-retrain-gpt-2-117m-to-generate-custom-text-content-8bb5363d8b7f)

[Notebook Guide](https://github.com/ak9250/gpt-2-colab/blob/master/GPT_2.ipynb)

[Library](https://github.com/nshepperd/gpt-2)

# Installing and Setting up Environment

In [1]:
import os

In [2]:
!git clone https://github.com/nshepperd/gpt-2.git

fatal: destination path 'gpt-2' already exists and is not an empty directory.


In [3]:
os.chdir("gpt-2")
!pip3 install tensorflow==1.15.0
!pip install -r requirements.txt

In [4]:
!python3 download_model.py 345M
!export PYTHONIOENCODING=UTF-8

Fetching checkpoint: 1.00kit [00:00, 803kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 54.8Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 823kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:21, 65.7Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 8.34Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 56.2Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 50.6Mit/s]                                                       


In [5]:
os.chdir("src")

In [6]:
import json
import numpy as np
import tensorflow as tf
import model, sample, encoder
import pandas as pd
import numpy as np

# Loading Training Data

In [7]:
!curl -L -O https://raw.githubusercontent.com/aditeyabaral/gpt2-implementation/main/Simpsons.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8998k  100 8998k    0     0  12.0M      0 --:--:-- --:--:-- --:--:-- 11.9M


In [8]:
df = pd.read_csv("Simpsons.csv")
df.dropna(inplace = True)
df.drop_duplicates(inplace = True)
df.reset_index(inplace = True)
df.drop(columns = "index", inplace = True)
print(df.shape)
df.head()

(126646, 2)


,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [9]:
def character_slice(character):
  if character in np.unique(df[["raw_character_text"]]):
    return df[df["raw_character_text"] == character]

In [10]:
CHARACTER = "Lisa Simpson"

character_df = character_slice(CHARACTER)
print(character_df.shape)
character_df.head()

(10144, 2)


,raw_character_text,spoken_words
1,Lisa Simpson,Where's Mr. Bergstrom?
3,Lisa Simpson,That life is worth living.
8,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!
10,Lisa Simpson,Do you know where I could find him?
12,Lisa Simpson,"The train, how like him... traditional, yet en..."


## Adding delimiters

This is optional - allows for retention of training data structure

In [11]:
train_data = "\n<|endoftext|>\n".join(character_df["spoken_words"].values)

In [12]:
os.chdir("..")
with open("corpus.txt","w") as f:
  f.write(train_data)

# Training GPT-2

This is an endless loop, terminate execution to stop process

In [ ]:
!PYTHONPATH=src /content/gpt-2/train.py --dataset /content/gpt-2/src/corpus.txt --model_name '345M'


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2020-10-20 09:48:35.338880: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-10-20 09:48:35.343637: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-10-20 09:48:35.343888: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1633480 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-10-20 09:48:35.343922: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-10-20 09:48:35.345913

# Generation

## User Prompt

In [ ]:
!python3 src/interactive_conditional_samples.py --top_k 40 --model_name "345M"

## Random Sampling

In [ ]:
!python3 src/generate_unconditional_samples.py --model_name "345M" | tee /tmp/samples